In [1]:
import numpy as np
import pandas as pd
from keras.models import Model, load_model

Using Theano backend.


# Training

In [ ]:
# TODO

# Predicting

In [8]:
start_date = np.datetime64("2020-08-01")
end_date = np.datetime64("2020-08-04")

## Load candidate model

In [2]:
model_file = "tests/fixtures/20200731_predictor.h5"

In [3]:
from keras.constraints import Constraint

class Positive(Constraint):

    def __call__(self, w):
        return K.abs(w)

In [5]:
predictor_model = load_model(model_file, custom_objects={"Positive": Positive})

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/keras/layers/core.py:690: UserWarning: `output_shape` argument not specified for layer prediction and cannot be automatically inferred with the Theano backend. Defaulting to output shape `[(None, 1), (None, 1)]` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


## Load NPIs

In [6]:
LOCAL_DATA_URL = "tests/fixtures/OxCGRT_latest.csv"

In [18]:
# Reload the module to get the latest changes
import xprize_robojudge
from importlib import reload
reload(xprize_robojudge)
from xprize_robojudge import XPrizeRobojudge, NPI_COLUMNS

In [23]:
robojudge = XPrizeRobojudge()
npis_df = robojudge.get_npis(start_date, end_date, url=LOCAL_DATA_URL)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
npis_df.head()

,CountryName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
213,Aruba,2020-08-01,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
214,Aruba,2020-08-02,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
215,Aruba,2020-08-03,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
216,Aruba,2020-08-04,0.0,1.0,0.0,4.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
454,Afghanistan,2020-08-01,3.0,3.0,2.0,4.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0


## Make prediction

In [13]:
# Reload the module to get the latest changes
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

In [21]:
snapshot_df = robojudge.load_dataset(url=LOCAL_DATA_URL)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [22]:
predictor = XPrizePredictor(model_file, snapshot_df, NPI_COLUMNS)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/keras/layers/core.py:690: UserWarning: `output_shape` argument not specified for layer prediction and cannot be automatically inferred with the Theano backend. Defaulting to output shape `[(None, 1), (None, 1)]` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [25]:
preds = predictor.submission_predict(start_date, end_date, npis_df)

In [26]:
preds.head()

,CountryName,Date,PredictedDailyNewCases
0,Aruba,2020-08-01,80.933947
1,Aruba,2020-08-02,68.922970
2,Aruba,2020-08-03,32.827954
3,Aruba,2020-08-04,57.897908
4,Afghanistan,2020-08-01,31.412385
